<a href="https://colab.research.google.com/github/vmatiasw/modelos_y_simulacion/blob/main/P4E4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [253]:
import pandas as pd
from collections import Counter
import random
import time

FUNCION_DE_PROBABILIDAD = {i+1: p for i, p in enumerate([0.11, 0.14, 0.09, 0.08, 0.12, 0.10, 0.09, 0.07, 0.11, 0.09])}
SOPORTE = FUNCION_DE_PROBABILIDAD.keys()
print(FUNCION_DE_PROBABILIDAD)

def medir_tiempo(func):
    inicio = time.perf_counter()
    resultado = func()
    fin = time.perf_counter()
    return resultado, fin - inicio

def obtener_DataFrame(funcion_de_probabilidad, P):
  data = {
      "X": list(funcion_de_probabilidad.keys()),
      "P(X) Real": [P[x] for x in funcion_de_probabilidad.keys()],
      "Estimación": list(funcion_de_probabilidad.values()),
      "Diferencia": [abs(P[x] - funcion_de_probabilidad[x]) for x in funcion_de_probabilidad.keys()]
  }
  return pd.DataFrame(data)

def estimar_funcion_de_probabilidad(p, soporte, n):
  muestras = [p() for _ in range(n)]
  frecuencias = Counter(muestras)

  funcion_de_probabilidad = {x: frecuencias.get(x, 0) / n for x in soporte}

  return funcion_de_probabilidad

def calcular_TVD(f_estimada, f_real):
  """
  Calcula la Distancia Total de Variación (TVD) entre dos distribuciones discretas.
  Es una métrica entre 0 y 1 que mide la diferencia total entre probabilidades.

  Se interpreta como el máximo error total posible al usar la distribución estimada
  en lugar de la distribución real.
  """
  return 0.5 * sum(abs(f_real[x] - f_estimada.get(x, 0)) for x in f_real)

def calificar_generador(generador_de_muestras):
  n = 100_000
  f_estimada, tiempo = medir_tiempo(lambda: estimar_funcion_de_probabilidad(generador_de_muestras, SOPORTE, n))
  print(f"Tiempo de ejecución: {tiempo/n:.6f} segundos")
  print(obtener_DataFrame(f_estimada, FUNCION_DE_PROBABILIDAD))
  tvd = calcular_TVD(f_estimada, FUNCION_DE_PROBABILIDAD)
  print(f"Distancia total de variación (TVD): {tvd:.6f}")


{1: 0.11, 2: 0.14, 3: 0.09, 4: 0.08, 5: 0.12, 6: 0.1, 7: 0.09, 8: 0.07, 9: 0.11, 10: 0.09}


In [277]:
# a) Método de rechazo con una uniforme discreta, buscando la cota c más baja posible.
QY = 1/10

def AyR(p, q, c):
  """
  Método de aceptación-rechazo para generar muestras de la distribución `p` usando una distribución propuesta `q` y una constante `c`.

  Parámetros:
  p (list/dict): Distribución de probabilidad, donde `p[y]` es la probabilidad asociada a un valor `y`.
  q (function): Función que genera un valor aleatorio `y` según la distribución propuesta.
  c (float): Constante de escala que ajusta la relación entre `p` y `q`.

  Devuelve:
  int: Un valor `y` del soporte de `p` basado en la probabilidad ajustada por `q` y `c`.
  """
  while True:
    u = random.random()
    y = q()
    py = p[y]
    if u < py/(QY*c):
      return y

def q():
  u = random.random()
  return int(u*len(FUNCION_DE_PROBABILIDAD)+1) # cada valor con prob 1/10;

generador_de_muestras = lambda: AyR(FUNCION_DE_PROBABILIDAD, q, 1.4)
calificar_generador(generador_de_muestras)

Tiempo de ejecución: 0.000001 segundos
    X  P(X) Real  Estimación  Diferencia
0   1       0.11     0.10911     0.00089
1   2       0.14     0.14020     0.00020
2   3       0.09     0.08927     0.00073
3   4       0.08     0.08088     0.00088
4   5       0.12     0.11863     0.00137
5   6       0.10     0.10051     0.00051
6   7       0.09     0.09055     0.00055
7   8       0.07     0.07115     0.00115
8   9       0.11     0.10988     0.00012
9  10       0.09     0.08982     0.00018
Distancia total de variación (TVD): 0.003290


In [269]:
# b) Método de rechazo con una uniforme discreta, usando c = 3.

generador_de_muestras = lambda: AyR(FUNCION_DE_PROBABILIDAD, q, 3)
calificar_generador(generador_de_muestras)

Tiempo de ejecución: 0.000003 segundos
    X  P(X) Real  Estimación  Diferencia
0   1       0.11     0.11244     0.00244
1   2       0.14     0.13739     0.00261
2   3       0.09     0.09025     0.00025
3   4       0.08     0.08044     0.00044
4   5       0.12     0.11964     0.00036
5   6       0.10     0.10137     0.00137
6   7       0.09     0.08850     0.00150
7   8       0.07     0.06914     0.00086
8   9       0.11     0.11146     0.00146
9  10       0.09     0.08937     0.00063
Distancia total de variación (TVD): 0.005960


In [283]:
# c) Transformada inversa.

def funcion_distribucion_acumulada(f):
  """
  Recibe una funcion de probabilidad de masa en forma de diccionario de pares
  (valor, probabilidad) y devuelve su funcion de distribucion acumulada.
  Solo definidas para el soporte.
  """
  F = {}
  suma = 0
  for valor, probabilidad in f.items():
    suma += probabilidad
    F[valor] = suma
  return F

def ordenar_f(f):
  return dict(sorted(f.items(), key=lambda x: x[1], reverse=True))

def TI(F):
  u = random.random()
  for v, p in F.items():
    if u < p:
      return v

FDA = funcion_distribucion_acumulada(ordenar_f(FUNCION_DE_PROBABILIDAD))
generador_de_muestras = lambda: TI(FDA)
calificar_generador(generador_de_muestras)

Tiempo de ejecución: 0.000001 segundos
    X  P(X) Real  Estimación  Diferencia
0   1       0.11     0.11210     0.00210
1   2       0.14     0.14168     0.00168
2   3       0.09     0.09015     0.00015
3   4       0.08     0.07963     0.00037
4   5       0.12     0.11941     0.00059
5   6       0.10     0.09984     0.00016
6   7       0.09     0.08880     0.00120
7   8       0.07     0.07004     0.00004
8   9       0.11     0.10871     0.00129
9  10       0.09     0.08964     0.00036
Distancia total de variación (TVD): 0.003970


In [264]:
# d) Método de la urna: utilizar un arreglo A de tamaño 100 donde cada valor i
# está en exactamente p[i] ∗ 100 posiciones. El método debe devolver A[k] con
# probabilidad 0,01. ¿Por qué funciona? -> las probabilidades son de a o sumo
# dos decimales y su suma da 1.

def generar_urna(funcion_de_probabilidad):
    urna = []
    for v, p in funcion_de_probabilidad.items():
        urna.extend([v] * int(p * 100))
    return urna

urna = generar_urna(FUNCION_DE_PROBABILIDAD)
generador_de_muestras = lambda: urna[int(random.random()*len(urna))]
calificar_generador(generador_de_muestras)

Tiempo de ejecución: 0.000001 segundos
    X  P(X) Real  Estimación  Diferencia
0   1       0.11     0.10997     0.00003
1   2       0.14     0.14156     0.00156
2   3       0.09     0.09064     0.00064
3   4       0.08     0.07953     0.00047
4   5       0.12     0.11914     0.00086
5   6       0.10     0.09862     0.00138
6   7       0.09     0.09095     0.00095
7   8       0.07     0.06953     0.00047
8   9       0.11     0.11003     0.00003
9  10       0.09     0.09003     0.00003
Distancia total de variación (TVD): 0.003210
